# LCA without foreground

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_exiobase.background_importer import *
from bw_exiobase.foreground_importer import *
from bw_exiobase.datapackage_builder import *
from bw_exiobase.uncertainty_handler import *
from notebooks.NAME_exiobase_lca import *
from constants import *

1. Get technosphere matrix

In [2]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [3]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [4]:
cf_matrix = bg_importer.form_cf_matrix(EMISSION_CODE_FILE, METHOD, GHG)

4. Add foreground system with 1 columns

In [5]:
fg_df = pd.read_csv(FG_FILE, delimiter=",")
fg_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


In [6]:
fg_activities = get_fg_activities(FG_FILE, ",")
fg_activities

['column_1']

In [7]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline',
 'EU28-Gas/Diesel Oil',
 'EU28-Refinery Feedstocks',
 'EU28-Lubricants',
 'EU28-P- and other fertiliser',
 'EU28-Chemicals nec',
 'EU28-Biogasoline',
 'EU28-Biodiesels',
 'EU28-Cement, lime and plaster',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'EU28-Aluminium and aluminium products',
 'EU28-Copper products',
 'EU28-Electricity by coal',
 'EU28-Electricity by gas',
 'EU28-Electricity by nuclear',
 'EU28-Electricity by hydro',
 'EU28-Electricity by wind',
 'EU28-Electricity by petroleum and other oil derivatives',
 'EU28-Electricity by biomass and waste',
 'EU28-Electricity by solar photovoltaic',
 'EU28-Electricity by solar thermal',
 'EU28-Electricity by tide, wave, ocean',
 'EU28-Electricity by Geothermal',
 'EU28-Electricity nec',
 'EU28-Steam and hot water supply 

In [8]:
print(tech_matrix)
print("---")
print(bio_matrix)

[[7.36338343e-01 1.32512793e-03 2.50369329e-03 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 [9.91365844e-03 8.30899707e-01 3.02023124e-02 ... 9.94126465e-05
  5.41113056e-05 9.39990163e-05]
 [4.32464974e-05 2.21849241e-03 9.99194240e-01 ... 3.70894886e-08
  2.00023023e-08 7.04757337e-09]
 ...
 [6.34806533e-06 8.26437110e-06 7.47308387e-07 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [1.23636432e-05 1.94495460e-05 2.45543251e-06 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [6.07845068e-06 1.32530898e-05 1.91153928e-06 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[6.92719535e+04 2.24902521e+05 1.58717207e+06 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [1.11657045e+02 3.26772076e+02 1.45794545e+01 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [5.76029465e+01 6.48498046e+01 6.86348286e+00 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [2.19718421e+00 3.01196685e+00 3.69253098e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

In [9]:
fg_importer = ForegroundImporter()
bgfg, fgfg, fgbg, fgbi = fg_importer.extend_matrix_2(fg_df, GHG, fg_activities, bg_activities)
tech_matrix, bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, bgfg, fgfg, fgbg, fgbi)
print(tech_matrix)
print("---")
print(bio_matrix)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.36338343e-01 1.32512793e-03 ... 1.21134511e-04
  7.62826166e-05 4.74480856e-04]
 [           nan 9.91365844e-03 8.30899707e-01 ... 9.94126465e-05
  5.41113056e-05 9.39990163e-05]
 ...
 [0.00000000e+00 6.34806533e-06 8.26437110e-06 ... 9.86260390e-01
  1.12285354e-04 3.46522374e-03]
 [0.00000000e+00 1.23636432e-05 1.94495460e-05 ... 5.76682882e-04
  9.87318876e-01 2.70409581e-03]
 [0.00000000e+00 6.07845068e-06 1.32530898e-05 ... 6.24171469e-03
  9.40994608e-04 9.95178298e-01]]
---
[[0.00000000e+00 6.92719535e+04 2.24902521e+05 ... 8.11463102e+04
  5.09153559e+04 5.02862192e+04]
 [0.00000000e+00 1.11657045e+02 3.26772076e+02 ... 4.06909320e+03
  1.11515389e+03 4.31986073e+03]
 [0.00000000e+00 5.76029465e+01 6.48498046e+01 ... 1.11483525e+03
  3.28857868e+02 1.21602199e+03]
 ...
 [0.00000000e+00 2.19718421e+00 3.01196685e+00 ... 1.38856482e+00
  6.37694131e-01 7.10205514e

5. Build datapackage

In [10]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_bw_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

6. Run static simulation

In [11]:
mysector = bg_activities.index('RoW-Steam and hot water supply services')

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")


Brightway calculated lca score: (nan, 'RoW-Steam and hot water supply services')


7. Run pedigree uncertainty simulation